In [142]:
import cv2
import numpy as np
import pandas as pd
import requests

In [143]:
df = pd.read_excel("excel/skipsListe.xlsx")
imo_column = df["imo"]
imo_list = list(imo_column)

In [144]:
def load_image_by_imo(imo):
    image_url = "https://www.ship-info.com/Vessels/" + str(imo) + ".jpg" # Replace with your image URL
    response = requests.get(image_url)
    if response.status_code != 200: return None
    image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    return image

In [145]:
from detector import Detector
detector = Detector("models/excavatorV8s.pt")
classifier_dictonary = {}
#classifier_list = []
print("")
for imo in imo_list[0:10]:
    image = load_image_by_imo(imo)
    if image is not None: classifier_dictonary[imo] = detector.find_objects_image(image)
    else: classifier_dictonary[imo] = [("No image found",0)] #could not load image



   
    


0: 384x640 (no detections), 154.0ms
Speed: 6.0ms preprocess, 154.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Gravemaskin, 145.0ms
Speed: 2.0ms preprocess, 145.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 144.9ms
Speed: 3.0ms preprocess, 144.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 141.0ms
Speed: 3.0ms preprocess, 141.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 146.0ms
Speed: 3.0ms preprocess, 146.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Gravemaskin, 141.0ms
Speed: 2.0ms preprocess, 141.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Gravemaskin, 149.0ms
Speed: 2.0ms preprocess, 149.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 147.0ms
Speed: 2.0ms preprocess,

In [147]:
df["classes"] = ["None"]*(df.shape[0])
df["confidence"] = [0]*(df.shape[0])
#classifier_list = list(classifier_dictonary.values()) # we do not care about the imo

for imo, array in classifier_dictonary.items():
    if len(array) == 0: continue #nothing classified
    class_string = ""
    confidence_string = ""
    for tuple in array:
        class_string += tuple[0] + " "
        confidence_string += str(round((tuple[1]),2)) + " "
    #df["classes"][i] = class_string
    df.loc[df["imo"] == imo, "classes"] = class_string
    df.loc[df["imo"] == imo, "confidence"] = confidence_string
    #df["confidence"][i] = confidence_string
 


        



df.to_excel("excel/test.xlsx", index=False)